In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
trainDataDir = "dataset/two-sigma-connect-rental-listing-inquiries/trainNaN.json.zip"

In [3]:
trainData = pd.read_json(trainDataDir, convert_dates=['created'])
trainData = trainData.reset_index(drop=True)

## Reformat display and street address

In [4]:
trainData = trainData.dropna(subset=['display_address','street_address'])
trainData

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
0,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
1,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low
2,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium
3,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium
4,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795,340 East 34th Street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49289,1.0,3,92bbbf38baadfde0576fc496bd41749c,2016-04-05 03:58:33,There is 700 square feet of recently renovated...,W 171 Street,"[Elevator, Dishwasher, Hardwood Floors]",40.8433,6824800,-73.9396,a61e21da3ba18c7a3d54cfdcc247e1f8,[https://photos.renthop.com/2/6824800_0682be16...,2800,620 W 171 Street,low
49290,1.0,2,5565db9b7cba3603834c4aa6f2950960,2016-04-02 02:25:31,"2 bedroom apartment with updated kitchen, rece...",Broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",40.8198,6813268,-73.9578,8f90e5e10e8a2d7cf997f016d89230eb,[https://photos.renthop.com/2/6813268_1e6fcc32...,2395,3333 Broadway,medium
49291,1.0,1,67997a128056ee1ed7d046bbb856e3c7,2016-04-26 05:42:03,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,210 Brighton 15th St,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",40.5765,6927093,-73.9554,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/6927093_93a52104...,1850,210 Brighton 15th St,medium
49292,1.0,2,3c0574a740154806c18bdf1fddd3d966,2016-04-19 02:47:33,Wonderful Bright Chelsea 2 Bedroom apartment o...,West 21st Street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",40.7448,6892816,-74.0017,c3cd45f4381ac371507090e9ffabea80,[https://photos.renthop.com/2/6892816_1a8d087a...,4195,350 West 21st Street,medium


In [5]:
#https://stackoverflow.com/questions/57503786/python-replace-abbreviation-in-text
replacers = {
    'st': 'street',
    'e': 'east',
    'w': 'west',
    's': 'south',
    'n': 'north',
    'ave': 'avenue',
    'rd': 'road'
}
trainData['street_address'] = (
    trainData['street_address']
    .str.lower()
    .str.replace('[.,]',' ')
    .str.split()
    .apply(lambda x: ' '.join([replacers.get(e, e) for e in x]))
)
trainData['display_address'] = (
    trainData['display_address']
    .str.lower()
    .str.replace('[.,]',' ')
    .str.split()
    .apply(lambda x: ' '.join([replacers.get(e, e) for e in x]))
)
trainData

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
0,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 borinquen place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 borinquen place,medium
1,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,east 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 east 44th,low
2,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,east 56th street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 east 56th street,medium
3,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,metropolitan avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 metropolitan avenue,medium
4,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,east 34th street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795,340 east 34th street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49289,1.0,3,92bbbf38baadfde0576fc496bd41749c,2016-04-05 03:58:33,There is 700 square feet of recently renovated...,west 171 street,"[Elevator, Dishwasher, Hardwood Floors]",40.8433,6824800,-73.9396,a61e21da3ba18c7a3d54cfdcc247e1f8,[https://photos.renthop.com/2/6824800_0682be16...,2800,620 west 171 street,low
49290,1.0,2,5565db9b7cba3603834c4aa6f2950960,2016-04-02 02:25:31,"2 bedroom apartment with updated kitchen, rece...",broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",40.8198,6813268,-73.9578,8f90e5e10e8a2d7cf997f016d89230eb,[https://photos.renthop.com/2/6813268_1e6fcc32...,2395,3333 broadway,medium
49291,1.0,1,67997a128056ee1ed7d046bbb856e3c7,2016-04-26 05:42:03,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,210 brighton 15th street,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",40.5765,6927093,-73.9554,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/6927093_93a52104...,1850,210 brighton 15th street,medium
49292,1.0,2,3c0574a740154806c18bdf1fddd3d966,2016-04-19 02:47:33,Wonderful Bright Chelsea 2 Bedroom apartment o...,west 21st street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",40.7448,6892816,-74.0017,c3cd45f4381ac371507090e9ffabea80,[https://photos.renthop.com/2/6892816_1a8d087a...,4195,350 west 21st street,medium


# Impute missing building id

In [6]:
#https://stackoverflow.com/questions/27159189/find-empty-or-nan-entry-in-pandas-dataframe
buildingNAN = trainData.loc[trainData['building_id'].isnull()]
buildingNAN.head(4)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
19,1.0,1,None,2016-06-25 01:35:36,"DISCLAIMER: Photos are of similar unit. Great,...",main street,[Doorman],40.7621,7213185,-73.9486,e6ecf676c4359ad2903283834f1fd301,[https://photos.renthop.com/2/7213185_9102c3f2...,2495,576 main street,low
26,1.0,2,None,2016-06-05 11:31:08,Find your place in this thoughtfully designed ...,cliff street,"[Common Outdoor Space, Laundry In Unit, Doorma...",40.7084,7114220,-74.0048,26d69329a0f3b9abecbddd6abcb5b622,[https://photos.renthop.com/2/7114220_1246ad3d...,5425,cliff street,low
38,2.0,3,None,2016-06-24 07:55:11,Welcome to your new home! This extravagantly o...,president street,"[Dining Room, Laundry in Building, High Speed ...",40.6678,7211226,-73.9398,929baf387cf71d98482a9743ab8a0890,[https://photos.renthop.com/2/7211226_7c7b8bc2...,3050,1475 president street,low
43,1.0,1,None,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street,low


In [7]:
trainDataNonNAN = trainData.dropna(subset=['building_id'])
trainDataNonNAN['building_id'].isnull().any()

False

In [8]:
buildingID = trainDataNonNAN[['building_id','latitude','longitude','display_address', 'street_address']].dropna()

In [9]:
a = buildingNAN.merge(buildingID, how='inner', on=['latitude','longitude','display_address', 'street_address'])
a.head()

,bathrooms,bedrooms,building_id_x,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,building_id_y
0,2.0,3,None,2016-06-24 07:55:11,Welcome to your new home! This extravagantly o...,president street,"[Dining Room, Laundry in Building, High Speed ...",40.6678,7211226,-73.9398,929baf387cf71d98482a9743ab8a0890,[https://photos.renthop.com/2/7211226_7c7b8bc2...,3050,1475 president street,low,b66d3340ba269e6f184bfad550a6f05f
1,2.0,3,None,2016-04-29 03:29:11,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",president street,[],40.6678,6941286,-73.9398,a12c733c6cba53a6d2fe8a1fdfd5a674,[https://photos.renthop.com/2/6941286_252212fd...,3100,1475 president street,low,b66d3340ba269e6f184bfad550a6f05f
2,1.0,1,None,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street,low,7e9fd0dea8ad2c8ac0a1022a1f0d997b
3,1.0,1,None,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street,low,7e9fd0dea8ad2c8ac0a1022a1f0d997b
4,1.0,1,None,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street,low,7e9fd0dea8ad2c8ac0a1022a1f0d997b


In [10]:
a = a.drop_duplicates(subset=['listing_id']).drop('building_id_x',axis=1).rename(columns ={'building_id_y':'building_id'})
a

,bathrooms,bedrooms,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,building_id
0,2.0,3,2016-06-24 07:55:11,Welcome to your new home! This extravagantly o...,president street,"[Dining Room, Laundry in Building, High Speed ...",40.6678,7211226,-73.9398,929baf387cf71d98482a9743ab8a0890,[https://photos.renthop.com/2/7211226_7c7b8bc2...,3050,1475 president street,low,b66d3340ba269e6f184bfad550a6f05f
1,2.0,3,2016-04-29 03:29:11,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",president street,[],40.6678,6941286,-73.9398,a12c733c6cba53a6d2fe8a1fdfd5a674,[https://photos.renthop.com/2/6941286_252212fd...,3100,1475 president street,low,b66d3340ba269e6f184bfad550a6f05f
2,1.0,1,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street,low,7e9fd0dea8ad2c8ac0a1022a1f0d997b
17,1.0,3,2016-06-08 01:12:34,"Prime East Village 3 bedroom, 1 bath apartment...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7287,7122279,-73.9810,e7f859b81a2a75ac93042a721d70a0aa,[https://photos.renthop.com/2/7122279_fea77391...,4200,504 east 12th street,low,7e9fd0dea8ad2c8ac0a1022a1f0d997b
32,1.0,1,2016-06-11 01:27:45,Welcome to your next new home.This 1BR/1BA uni...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7287,7138528,-73.9810,89179891e92a87e72ac3132fc5a977e8,[https://photos.renthop.com/2/7138528_940f2e70...,2850,504 east 12th street,low,7e9fd0dea8ad2c8ac0a1022a1f0d997b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95414,1.0,2,2016-04-13 05:11:26,Luxury New 1/1 w/balcony,stockholm street,"[Cats Allowed, Dogs Allowed, Exclusive]",40.6980,6866700,-73.9243,62826f3ae01f2ddc93b9cd28c659ab2b,None,3100,113 stockholm street,low,15d8135ac5030fb076812afce8d75998
95417,1.0,1,2016-04-22 01:25:17,This lovely renovated one bedroom has a large ...,east 81st street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7744,6909433,-73.9531,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6909433_53b0228d...,2595,313 east 81st street,low,bc44310f4b6528e1b36066359d8941f7
95421,1.0,1,2016-04-06 01:29:35,This is a Inwood great one bedroom apartment t...,sherman avenue,[Pre-War],40.8645,6828115,-73.9246,9153d5678d7731f9ae283aba33932c1e,[https://photos.renthop.com/2/6828115_7b50f982...,1325,124 sherman avenue,low,394d115a27afc416df7af4d93dbc0d23
95423,1.0,1,2016-04-09 03:12:32,"Roof top Pool this Summer? You Got It! Luxury,...",east 84th street,"[Swimming Pool, Doorman, Elevator, Terrace, La...",40.7758,6847952,-73.9506,7977d30107b81a8888b95ae909448bbe,[https://photos.renthop.com/2/6847952_abe45fa3...,3650,351 east 84th street,low,5bb6a1450ca05310606a43154cef1aba


In [11]:
trainData = a.append(trainDataNonNAN,ignore_index = True, sort=True)
trainData

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-06-24 07:55:11,Welcome to your new home! This extravagantly o...,president street,"[Dining Room, Laundry in Building, High Speed ...",low,40.6678,7211226,-73.9398,929baf387cf71d98482a9743ab8a0890,[https://photos.renthop.com/2/7211226_7c7b8bc2...,3050,1475 president street
1,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-04-29 03:29:11,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",president street,[],low,40.6678,6941286,-73.9398,a12c733c6cba53a6d2fe8a1fdfd5a674,[https://photos.renthop.com/2/6941286_252212fd...,3100,1475 president street
2,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street
3,1.0,3,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-08 01:12:34,"Prime East Village 3 bedroom, 1 bath apartment...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7122279,-73.9810,e7f859b81a2a75ac93042a721d70a0aa,[https://photos.renthop.com/2/7122279_fea77391...,4200,504 east 12th street
4,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-11 01:27:45,Welcome to your next new home.This 1BR/1BA uni...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7138528,-73.9810,89179891e92a87e72ac3132fc5a977e8,[https://photos.renthop.com/2/7138528_940f2e70...,2850,504 east 12th street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46158,1.0,3,92bbbf38baadfde0576fc496bd41749c,2016-04-05 03:58:33,There is 700 square feet of recently renovated...,west 171 street,"[Elevator, Dishwasher, Hardwood Floors]",low,40.8433,6824800,-73.9396,a61e21da3ba18c7a3d54cfdcc247e1f8,[https://photos.renthop.com/2/6824800_0682be16...,2800,620 west 171 street
46159,1.0,2,5565db9b7cba3603834c4aa6f2950960,2016-04-02 02:25:31,"2 bedroom apartment with updated kitchen, rece...",broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.8198,6813268,-73.9578,8f90e5e10e8a2d7cf997f016d89230eb,[https://photos.renthop.com/2/6813268_1e6fcc32...,2395,3333 broadway
46160,1.0,1,67997a128056ee1ed7d046bbb856e3c7,2016-04-26 05:42:03,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,210 brighton 15th street,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",medium,40.5765,6927093,-73.9554,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/6927093_93a52104...,1850,210 brighton 15th street
46161,1.0,2,3c0574a740154806c18bdf1fddd3d966,2016-04-19 02:47:33,Wonderful Bright Chelsea 2 Bedroom apartment o...,west 21st street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",medium,40.7448,6892816,-74.0017,c3cd45f4381ac371507090e9ffabea80,[https://photos.renthop.com/2/6892816_1a8d087a...,4195,350 west 21st street


# Drop missing value in remaining columns

In [12]:
trainDataNoPhoto = trainData.dropna(subset=['display_address','latitude','longitude','street_address']).reset_index(drop=True)
trainDataNoPhoto

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-06-24 07:55:11,Welcome to your new home! This extravagantly o...,president street,"[Dining Room, Laundry in Building, High Speed ...",low,40.6678,7211226,-73.9398,929baf387cf71d98482a9743ab8a0890,[https://photos.renthop.com/2/7211226_7c7b8bc2...,3050,1475 president street
1,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-04-29 03:29:11,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",president street,[],low,40.6678,6941286,-73.9398,a12c733c6cba53a6d2fe8a1fdfd5a674,[https://photos.renthop.com/2/6941286_252212fd...,3100,1475 president street
2,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street
3,1.0,3,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-08 01:12:34,"Prime East Village 3 bedroom, 1 bath apartment...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7122279,-73.9810,e7f859b81a2a75ac93042a721d70a0aa,[https://photos.renthop.com/2/7122279_fea77391...,4200,504 east 12th street
4,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-11 01:27:45,Welcome to your next new home.This 1BR/1BA uni...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7138528,-73.9810,89179891e92a87e72ac3132fc5a977e8,[https://photos.renthop.com/2/7138528_940f2e70...,2850,504 east 12th street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46148,1.0,3,92bbbf38baadfde0576fc496bd41749c,2016-04-05 03:58:33,There is 700 square feet of recently renovated...,west 171 street,"[Elevator, Dishwasher, Hardwood Floors]",low,40.8433,6824800,-73.9396,a61e21da3ba18c7a3d54cfdcc247e1f8,[https://photos.renthop.com/2/6824800_0682be16...,2800,620 west 171 street
46149,1.0,2,5565db9b7cba3603834c4aa6f2950960,2016-04-02 02:25:31,"2 bedroom apartment with updated kitchen, rece...",broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.8198,6813268,-73.9578,8f90e5e10e8a2d7cf997f016d89230eb,[https://photos.renthop.com/2/6813268_1e6fcc32...,2395,3333 broadway
46150,1.0,1,67997a128056ee1ed7d046bbb856e3c7,2016-04-26 05:42:03,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,210 brighton 15th street,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",medium,40.5765,6927093,-73.9554,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/6927093_93a52104...,1850,210 brighton 15th street
46151,1.0,2,3c0574a740154806c18bdf1fddd3d966,2016-04-19 02:47:33,Wonderful Bright Chelsea 2 Bedroom apartment o...,west 21st street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",medium,40.7448,6892816,-74.0017,c3cd45f4381ac371507090e9ffabea80,[https://photos.renthop.com/2/6892816_1a8d087a...,4195,350 west 21st street


In [13]:
trainDataNoPhoto.isnull().any()

bathrooms          False
bedrooms           False
building_id        False
created            False
description        False
display_address    False
features           False
interest_level     False
latitude           False
listing_id         False
longitude          False
manager_id         False
photos              True
price              False
street_address     False
dtype: bool

In [14]:
trainDataWithoutNaNDir = "dataset/two-sigma-connect-rental-listing-inquiries/trainCleaned.json.zip"
trainDataNoPhoto.to_json(trainDataWithoutNaNDir)